## CSV Data to Delta Tables (Fact Tables)

#### 1. Configuraciones Previas

In [1]:
# Importar librerias
import os
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from delta.tables import DeltaTable
from pyspark.sql.functions import col, lit, when
from datetime import date

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 3, Finished, Available, Finished)

In [2]:
# Ejecutar sesiones de spark
spark = SparkSession.builder.appName("PerezLLorca").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 4, Finished, Available, Finished)

In [3]:
# Declarar el directorio de ficheros csv de dimensiones en Landing
landing_dim_files = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/hechos/'
# Acceso al sistema de ficheros Hadoop
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(landing_dim_files))

# Lista con los nombres de los ficheros CSV
csv_files = [f.getPath().getName() for f in status if f.getPath().getName().endswith(".csv")]

print("Ficheros CSV encontrados:")
print(csv_files)

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 5, Finished, Available, Finished)

Ficheros CSV encontrados:
['HechosGastosDiario.csv', 'HechosOtrosGastosDiario.csv', 'HechosOtrosIngresosExplotacionDiario.csv', 'HechosOtrosIngresosNoOrdinariosDiario.csv', 'HechosVentasDiario.csv']


In [29]:
## Declarar nombres de tablas
# Tabla Dimensiones
nombre_tabla_HechosIngresosDiario = 'HechosIngresosDiario'
nombre_tabla_HechosOtrosGastosDiario = 'HechosGastosDiario'

## Declarar directorios
# Directorio ficheros dimension (Lakehouse de Landing)
landing_fact_HechosGastosDiario = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/hechos/HechosGastosDiario.csv'
landing_fact_HechosOtrosGastosDiario = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/hechos/HechosOtrosGastosDiario.csv'
landing_fact_HechosOtrosIngresosExplotacionDiario = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/hechos/HechosOtrosIngresosExplotacionDiario.csv'
landing_fact_HechosOtrosIngresosNoOrdinariosDiario = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/hechos/HechosOtrosIngresosNoOrdinariosDiario.csv'
landing_fact_HechosVentasDiario = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Files/Landing/hechos/HechosVentasDiario.csv'

# Directorio schema dbo
dbo_path = 'abfss://3ec8fad3-44ca-4524-9168-00acec9ab0e1@onelake.dfs.fabric.microsoft.com/57b6f2fa-a351-4c23-9297-4b2b26b28d55/Tables/dbo/'

# Directorio de las tablas a escribir en schema dbo
path_to_write_fact_mov_ingresos_diario = os.path.join(dbo_path, nombre_tabla_HechosIngresosDiario)
path_to_write_fact_mov_gastos_diario = os.path.join(dbo_path, nombre_tabla_HechosOtrosGastosDiario)


StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 31, Finished, Available, Finished)

#### 2. Lectura & Tratamiento de datos

##### 2.1 Lectura & Tratamiento de datos de Ingresos

In [14]:
# Carga de datos de la tabla HechosVentasDiario
df_HechosVentasDiario = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_fact_HechosVentasDiario)
df_HechosVentasDiario = df_HechosVentasDiario.withColumn("Grupo", lit("7"))
df_HechosVentasDiario = df_HechosVentasDiario.withColumn("Tipo", lit("Ingreso"))
df_HechosVentasDiario = df_HechosVentasDiario.select('Fecha', 'Ejercicio', 'Mes',  'Grupo', 'Tipo', 'CuentaContable', 'DescripcionCuenta', 'ClienteID', 'ProductoID', 'Importe')
# Visualizar los datos
display(df_HechosVentasDiario)

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 52e8ff25-64a8-4738-95a5-08ecf4696b46)

In [15]:
#df_HechosVentasDiario.columns

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 17, Finished, Available, Finished)

In [16]:
# Carga de datos de la tabla HechosOtrosIngresosNoOrdinariosDiario
df_HechosOtrosIngresosNoOrdinariosDiario = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_fact_HechosOtrosIngresosNoOrdinariosDiario)
df_HechosOtrosIngresosNoOrdinariosDiario = df_HechosOtrosIngresosNoOrdinariosDiario.withColumn("Grupo", lit("7"))
df_HechosOtrosIngresosNoOrdinariosDiario = df_HechosOtrosIngresosNoOrdinariosDiario.withColumn("Tipo", lit("Ingreso"))
df_HechosOtrosIngresosNoOrdinariosDiario = df_HechosOtrosIngresosNoOrdinariosDiario.select('Fecha', 'Ejercicio', 'Mes',  'Grupo', 'Tipo', 'CuentaContable', 'DescripcionCuenta', 'ClienteID', 'ProductoID', 'Importe')
# Visualizar los datos
display(df_HechosOtrosIngresosNoOrdinariosDiario)

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6ad5eb29-30a7-4cbf-ac82-7d9ca1966785)

In [17]:
# Carga de datos de la tabla HechosOtrosIngresosExplotacionDiario
df_HechosOtrosIngresosExplotacionDiario = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_fact_HechosOtrosIngresosExplotacionDiario)
df_HechosOtrosIngresosExplotacionDiario = df_HechosOtrosIngresosExplotacionDiario.withColumn("Grupo", lit("7"))
df_HechosOtrosIngresosExplotacionDiario = df_HechosOtrosIngresosExplotacionDiario.withColumn("Tipo", lit("Ingreso"))
df_HechosOtrosIngresosExplotacionDiario = df_HechosOtrosIngresosExplotacionDiario.select('Fecha', 'Ejercicio', 'Mes',  'Grupo', 'Tipo', 'CuentaContable', 'DescripcionCuenta', 'ClienteID', 'ProductoID', 'Importe')
# Visualizar los datos
display(df_HechosOtrosIngresosExplotacionDiario)

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a8adf830-a84b-4ad3-90d5-dc8d96657156)

In [18]:
# Obtención del df final para movimientos de ingresos
df_fact_mov_ingresos = df_HechosVentasDiario.union(df_HechosOtrosIngresosNoOrdinariosDiario).union(df_HechosOtrosIngresosExplotacionDiario)
display(df_fact_mov_ingresos)

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, efd00aab-1fc0-4804-b2dc-9ee2510c5935)

##### 2.2 Lectura & Tratamiento de datos de Gastos

In [30]:
# Carga de datos de la tabla HechosGastosDiario
df_HechosGastosDiario = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_fact_HechosGastosDiario)

# Visualizar los datos
display(df_HechosGastosDiario)

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 32, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 34758fd6-be24-41e2-a7e1-27dbd39f5c65)

In [31]:
# Carga de datos de la tabla HechosOtrosGastosDiario
df_HechosOtrosGastosDiario = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ",").csv(landing_fact_HechosOtrosGastosDiario)

# Visualizar los datos
display(df_HechosOtrosGastosDiario)

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 33, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4c9e6b94-4fe3-4899-8df8-0fbf7efba8b8)

In [32]:
# Obtención del df final para movimientos de gastos
df_fact_mov_gastos = df_HechosGastosDiario.union(df_HechosOtrosGastosDiario)
display(df_fact_mov_gastos)

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 34, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 19c77fe0-ddef-4ae7-8d1b-dcccb9d7bf6c)

#### 3. Escribir datos en Lakehouse (schema dbo)

In [27]:
# Escribir en Lakehouse contenido df_fact_mov_ingresos
df_fact_mov_ingresos.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_fact_mov_ingresos_diario)

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 29, Finished, Available, Finished)

In [33]:
# Escribir en Lakehouse contenido df_fact_mov_gastos
df_fact_mov_gastos.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(path_to_write_fact_mov_gastos_diario)

StatementMeta(, 0f5b6912-315d-456e-9ae5-d23b6e8d600d, 35, Finished, Available, Finished)